In [1]:
%pip install -q google-generativeai ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.7 MB/s eta 0:00:00


In [2]:
import os
import re
import json
import textwrap
import datetime
import google.generativeai as genai
from google.colab import userdata, files
from IPython.display import Markdown, display, HTML, Javascript
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Layout

In [3]:
def configure_api():
    try:
        api_key = userdata.get("GOOGLE_API_KEY")
        if not api_key:
            api_key = input("Digite sua chave API do Google Gemini: ")
        genai.configure(api_key=api_key)
        return True
    except Exception as e:
        print(f"❌ Erro ao configurar a API: {e}")
        return False

# Função auxiliar para exibir texto formatado em Markdown
def to_markdown(text):
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [4]:
def get_adaptive_styles():
    # CSS adaptativo que funciona tanto em tema claro quanto escuro
    return """
    <style>
    /* Estilos adaptativos para tema claro e escuro */
    .pdi-container {
        background: var(--colab-primary-surface-color, #ffffff);
        color: var(--colab-primary-text-color, #202124);
        border: 1px solid var(--colab-border-color, #dadce0);
        padding: 20px;
        border-radius: 12px;
        margin: 10px 0;
        box-shadow: 0 2px 8px rgba(0,0,0,0.1);
        font-family: 'Google Sans', Arial, sans-serif;
        line-height: 1.6;
    }

    .pdi-header {
        background: linear-gradient(135deg, #4285f4, #34a853);
        color: white;
        padding: 15px;
        border-radius: 12px;
        text-align: center;
        margin-bottom: 20px;
        box-shadow: 0 4px 12px rgba(66, 133, 244, 0.3);
    }

    .pdi-section {
        margin: 15px 0;
        padding: 12px;
        border-radius: 8px;
        background: var(--colab-secondary-surface-color, #f8f9fa);
        border-left: 4px solid var(--accent-color, #4285f4);
    }

    .emoji-title {
        font-weight: bold;
        font-size: 1.1em;
        margin-bottom: 8px;
        display: block;
    }

    .emoji-goal { color: #ea4335; }
    .emoji-resources { color: #4285f4; }
    .emoji-schedule { color: #34a853; }
    .emoji-skills { color: #9c27b0; }
    .emoji-tips { color: #ff9800; }
    .emoji-review { color: #00bcd4; }
    .emoji-current { color: #795548; }

    .pdi-list {
        margin: 8px 0;
        padding-left: 0;
        list-style: none;
    }

    .pdi-list li {
        margin: 5px 0;
        padding: 8px 12px;
        background: var(--colab-primary-surface-color, #ffffff);
        border-radius: 6px;
        border-left: 3px solid var(--accent-color, #4285f4);
        box-shadow: 0 1px 3px rgba(0,0,0,0.1);
    }

    .pdi-list li::before {
        content: "▸";
        color: var(--accent-color, #4285f4);
        font-weight: bold;
        margin-right: 8px;
    }

    .pdi-footer {
        text-align: center;
        margin-top: 20px;
        padding: 10px;
        font-size: 0.9em;
        color: var(--colab-secondary-text-color, #5f6368);
        border-top: 1px solid var(--colab-border-color, #dadce0);
    }

    .instructions-box {
        background: var(--colab-secondary-surface-color, #f8f9fa);
        color: var(--colab-primary-text-color, #202124);
        border: 1px solid var(--colab-border-color, #dadce0);
        padding: 15px;
        border-radius: 8px;
        margin-top: 20px;
    }

    /* Tema escuro específico */
    @media (prefers-color-scheme: dark) {
        .pdi-container {
            background: #1a1a1a;
            color: #e8eaed;
            border-color: #5f6368;
        }

        .pdi-section {
            background: #2d2d2d;
        }

        .pdi-list li {
            background: #1a1a1a;
            color: #e8eaed;
        }

        .instructions-box {
            background: #2d2d2d;
            color: #e8eaed;
            border-color: #5f6368;
        }
    }

    /* Para Colab específico em tema escuro */
    .theme-dark .pdi-container,
    [data-theme="dark"] .pdi-container {
        background: #1a1a1a !important;
        color: #e8eaed !important;
        border-color: #5f6368 !important;
    }

    .theme-dark .pdi-section,
    [data-theme="dark"] .pdi-section {
        background: #2d2d2d !important;
    }

    .theme-dark .pdi-list li,
    [data-theme="dark"] .pdi-list li {
        background: #1a1a1a !important;
        color: #e8eaed !important;
    }

    .theme-dark .instructions-box,
    [data-theme="dark"] .instructions-box {
        background: #2d2d2d !important;
        color: #e8eaed !important;
        border-color: #5f6368 !important;
    }
    </style>
    """

In [5]:
def format_pdi_with_html(pdi_text):
    # Dividir o texto em seções
    sections = pdi_text.split('\n\n')
    formatted_sections = []

    for section in sections:
        if not section.strip():
            continue

        lines = section.split('\n')
        if not lines:
            continue

        title_line = lines[0].strip()
        content_lines = lines[1:] if len(lines) > 1 else []

        # Identificar o tipo de seção pelo emoji
        section_class = ""
        if "🔍" in title_line:
            section_class = "emoji-current"
        elif "🎯" in title_line:
            section_class = "emoji-goal"
        elif "📚" in title_line:
            section_class = "emoji-resources"
        elif "📅" in title_line:
            section_class = "emoji-schedule"
        elif "🧩" in title_line:
            section_class = "emoji-tips"
        elif "🔁" in title_line:
            section_class = "emoji-review"

        # Formatar a seção
        formatted_title = f'<span class="emoji-title {section_class}">{title_line}</span>'

        # Processar conteúdo da seção
        if content_lines:
            content_html = []
            current_list = []

            for line in content_lines:
                line = line.strip()
                if not line:
                    continue

                if line.startswith('- ') or line.startswith('• '):
                    # Item de lista
                    item_text = line[2:].strip()
                    current_list.append(f'<li>{item_text}</li>')
                elif line.startswith('  ') and current_list:
                    # Sub-item (indentado)
                    sub_item = line.strip()
                    if sub_item.startswith('• '):
                        sub_item = sub_item[2:]
                    if current_list:
                        current_list[-1] = current_list[-1][:-5] + f'<br>&nbsp;&nbsp;{sub_item}</li>'
                else:
                    # Fechar lista anterior se existir
                    if current_list:
                        content_html.append('<ul class="pdi-list">' + ''.join(current_list) + '</ul>')
                        current_list = []

                    # Linha normal
                    if line:
                        content_html.append(f'<p>{line}</p>')

            # Fechar lista final se existir
            if current_list:
                content_html.append('<ul class="pdi-list">' + ''.join(current_list) + '</ul>')

            section_content = ''.join(content_html)
        else:
            section_content = ""

        formatted_section = f'''
        <div class="pdi-section">
            {formatted_title}
            {section_content}
        </div>
        '''
        formatted_sections.append(formatted_section)

    # Montar HTML final
    final_html = f"""
    {get_adaptive_styles()}
    <div class="pdi-container">
        <div class="pdi-header">
            <h2>📋 Plano de Desenvolvimento Individual (PDI)</h2>
            <p>Seu plano personalizado para alcançar seus objetivos</p>
        </div>

        {''.join(formatted_sections)}

        <div class="pdi-footer">
            🤖 Gerado em {datetime.datetime.now().strftime('%d/%m/%Y às %H:%M')} com IA Generativa
        </div>
    </div>
    """

    return HTML(final_html)

In [6]:
def gerar_pdi(area, habilidades, objetivo, horas, tempo_estimado, estilos):
    prompt = f"""
Crie um Plano de Desenvolvimento Individual (PDI) para alguém com o seguinte perfil:

🎯 Objetivo principal: {objetivo}
💡 Área de interesse: {area}
🔍 Habilidades atuais: {habilidades}
⏱️ Horas disponíveis por semana: {horas}
📆 Tempo estimado para alcançar o objetivo: {tempo_estimado}
🧠 Estilos de aprendizagem: {', '.join(estilos)}

O plano deve ser direto, fácil de entender, com formatação clara (sem markdown nem asteriscos), e incluir:

🎯 Planos de ação para alcançar o objetivo principal no tempo estimado
📚 Recursos recomendados separados por tipo (vídeos, livros, prática)
📅 Cronograma semanal baseado nas horas disponíveis e tempo estimado para alcançar o objetivo principal
🧩 Dicas para manter o foco
🔁 Estratégia de revisão e adaptação
⚠️ Incluir nomes de canais/livros, mas sem links
⚠️ Recursos recomendados, só exibir recursos recomendados com base na resposta de estilos de aprendizagem
⚠️ Faça recomendações e ações baseado no tempo que foi especificado, não sobrecarregando muito e nem deixando tempo de sobra, o ideal para o tempo especificado
🔍 Habilidades atuais: Liste e comente como essas habilidades podem ajudar na jornada, quebrando linhas na hora de escrever e isso deve ficar em primeiro lugar no PDI E DEVE TER EMOJI
⚠️ Pule linhas em habilidades atuais, não deixando tudo na mesma linha
⚠️ Evite parágrafos muito longos ou frases extensas. Cada ideia deve ocupar no máximo 2 a 3 linhas
🔁 Estratégia de revisão e adaptação deve ter quebra de linhas também
⚠️ Recomende pelo menos dois recursos especificados em estilos de aprendizagem

Formato exemplo:

🔍 Habilidades Atuais:
- Habilidade 1: como ela ajuda...
- Habilidade 2: como ela ajuda...
- Habilidade 3: como ela ajuda...

🎯 Planos de ação:
- Plano de ação 1: ...
- Plano de ação 2: ...
- Plano de ação 3: ...

📚 Recursos Recomendados:
- Tipo 1:
  • Recurso A: descrição
  • Recurso B: descrição
- Tipo 2:
  • Recurso C: descrição
  • Recurso D: descrição

📅 Cronograma semanal ({horas}h/semana):

Semana 1:
- 2h: ...
- 1h: ...
- 2h: ...

Semana 2:
- 2h: ...
- 1h: ...
- 2h: ...

E assim por diante...

🧩 Dicas para manter o foco:
- Dica 1: ...
- Dica 2: ...
- Dica 3: ...

🔁 Estratégia de revisão e adaptação:
- Estratégia 1: ...
- Estratégia 2: ...
- Estratégia 3: ...
"""
    try:
        # Usar um modelo mais avançado
        model = genai.GenerativeModel(model_name="gemini-1.5-flash")
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"Ocorreu um erro ao gerar o PDI: {str(e)}"

In [7]:
def salvar_pdi(pdi_text, dados_usuario):
    # Preparar dados para salvar
    nome_arquivo = f"PDI_{dados_usuario['area'].replace(' ', '_')}_{datetime.datetime.now().strftime('%Y%m%d_%H%M')}"

    # Salvar como texto
    with open(f"{nome_arquivo}.txt", "w", encoding="utf-8") as f:
        f.write(pdi_text)

    # Salvar também metadados em JSON
    dados_completos = {
        "pdi_texto": pdi_text,
        "metadados": dados_usuario,
        "data_geracao": datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }

    with open(f"{nome_arquivo}.json", "w", encoding="utf-8") as f:
        json.dump(dados_completos, f, ensure_ascii=False, indent=4)

    # Fazer download dos arquivos
    files.download(f"{nome_arquivo}.txt")
    files.download(f"{nome_arquivo}.json")

    return nome_arquivo

# Função para criar widgets com tema adaptativo
def create_adaptive_widget_style():
    return {
        'description_width': '150px'
    }

In [8]:
def criar_interface():
    # Configurar API primeiro
    if not configure_api():
        display(HTML("<div class='instructions-box'><p style='color:#ea4335; font-weight:bold'>⚠️ Por favor, configure a API do Google Gemini primeiro.</p></div>"))
        return

    # Estilo para os campos
    style = create_adaptive_widget_style()
    layout = Layout(width='85%')

    # Campos de entrada
    area_input = widgets.Text(description='💡 Área de interesse:', style=style, layout=layout)
    habilidades_input = widgets.Textarea(
        description='🔍 Habilidades atuais:',
        placeholder='Liste suas habilidades separadas por vírgula',
        style=style,
        layout=layout,
        rows=3
    )
    objetivo_input = widgets.Textarea(
        description='🎯 Objetivo principal:',
        placeholder='Descreva seu objetivo principal',
        style=style,
        layout=layout,
        rows=3
    )
    horas_input = widgets.IntSlider(
        description='⏱️ Horas por semana:',
        min=1,
        max=30,
        value=5,
        style=style,
        layout=layout
    )
    tempo_input = widgets.Text(
        description='📆 Tempo estimado:',
        placeholder='Ex: 2 meses, 3 semanas',
        style=style,
        layout=layout
    )

    # Checkboxes para estilos de aprendizagem
    estilos_options = [
        ('📹 Vídeos', 'vídeos'),
        ('📖 Leitura', 'leitura'),
        ('🛠️ Prática', 'prática'),
        ('🎧 Áudio', 'áudio'),
        ('📋 Projetos', 'projetos'),
        ('👥 Comunidades', 'comunidades')
    ]
    estilos_widget = widgets.SelectMultiple(
        options=estilos_options,
        description='🧠 Estilos:',
        style=style,
        layout=layout,
        rows=6
    )

    output = widgets.Output()
    status = widgets.HTML(value="")

    # Botões com melhor estilo
    gerar_btn = widgets.Button(
        description='🚀 Gerar PDI',
        button_style='primary',
        layout=Layout(width='200px', height='40px'),
        tooltip='Clique para gerar seu PDI personalizado'
    )

    salvar_btn = widgets.Button(
        description='💾 Salvar PDI',
        button_style='success',
        disabled=True,
        layout=Layout(width='200px', height='40px'),
        tooltip='Clique para baixar o PDI gerado'
    )

    # Variáveis para armazenar o PDI gerado
    pdi_gerado = {"texto": "", "dados": {}}

    # Função para validar os campos
    def validar_campos():
        if not area_input.value or not area_input.value.strip():
            return "⚠️ Por favor, preencha a área de interesse."
        if not habilidades_input.value or not habilidades_input.value.strip():
            return "⚠️ Por favor, liste suas habilidades atuais."
        if not objetivo_input.value or not objetivo_input.value.strip():
            return "⚠️ Por favor, defina seu objetivo principal."
        if not tempo_input.value or not tempo_input.value.strip():
            return "⚠️ Por favor, informe o tempo estimado."
        if not estilos_widget.value or len(estilos_widget.value) == 0:
            return "⚠️ Por favor, selecione pelo menos um estilo de aprendizagem."
        return None

    # Função para lidar com o clique no botão de gerar
    def on_gerar_click(b):
        # Limpar saída anterior
        output.clear_output()

        # Validar campos
        erro = validar_campos()
        if erro:
            status.value = f"<div class='instructions-box' style='border-left: 4px solid #ea4335;'>{erro}</div>"
            return

        status.value = "<div class='instructions-box' style='border-left: 4px solid #4285f4;'>⏳ Gerando seu PDI personalizado, por favor aguarde...</div>"

        # Coletar dados
        area = area_input.value
        habilidades = habilidades_input.value
        objetivo = objetivo_input.value
        horas = horas_input.value
        tempo_estimado = tempo_input.value
        estilos = list(estilos_widget.value)

        # Armazenar dados do usuário
        pdi_gerado["dados"] = {
            "area": area,
            "habilidades": habilidades,
            "objetivo": objetivo,
            "horas": horas,
            "tempo_estimado": tempo_estimado,
            "estilos": estilos
        }

        # Gerar PDI
        with output:
            try:
                # Gerar o PDI
                pdi_texto = gerar_pdi(area, habilidades, objetivo, horas, tempo_estimado, estilos)
                pdi_gerado["texto"] = pdi_texto

                # Exibir o PDI formatado
                display(format_pdi_with_html(pdi_texto))

                # Habilitar botão de salvar
                salvar_btn.disabled = False

                # Atualizar status
                status.value = "<div class='instructions-box' style='border-left: 4px solid #34a853;'>✅ PDI gerado com sucesso! Você pode salvá-lo agora.</div>"
            except Exception as e:
                status.value = f"<div class='instructions-box' style='border-left: 4px solid #ea4335;'>❌ Erro ao gerar o PDI: {str(e)}</div>"

    # Função para lidar com o clique no botão de salvar
    def on_salvar_click(b):
        if pdi_gerado["texto"]:
            try:
                nome_arquivo = salvar_pdi(pdi_gerado["texto"], pdi_gerado["dados"])
                status.value = f"<div class='instructions-box' style='border-left: 4px solid #34a853;'>✅ PDI salvo com sucesso como {nome_arquivo}.txt e {nome_arquivo}.json</div>"
            except Exception as e:
                status.value = f"<div class='instructions-box' style='border-left: 4px solid #ea4335;'>❌ Erro ao salvar o PDI: {str(e)}</div>"

    # Registrar callbacks
    gerar_btn.on_click(on_gerar_click)
    salvar_btn.on_click(on_salvar_click)

    # Header adaptativo
    header = HTML(f"""
    {get_adaptive_styles()}
    <div class="pdi-header">
        <h1>🎯 Gerador de Plano de Desenvolvimento Individual (PDI)</h1>
        <p>Crie um plano personalizado para alcançar seus objetivos profissionais com ajuda de IA</p>
    </div>
    """)

    # Exibir interface
    display(header)
    display(VBox([
        area_input,
        habilidades_input,
        objetivo_input,
        horas_input,
        tempo_input,
        widgets.HTML("<div style='margin: 10px 0; font-weight: bold;'>🧠 Selecione um ou mais estilos de aprendizagem:</div>"),
        estilos_widget,
        widgets.HTML("<div style='margin: 15px 0;'></div>"),  # Espaçamento
        HBox([gerar_btn, salvar_btn], layout=Layout(justify_content='center')),
        status,
        output
    ]))

In [9]:
criar_interface()
display(HTML(f"""
{get_adaptive_styles()}
<div class="instructions-box">
    <h3>📌 Instruções de Uso</h3>
    <ol style="line-height: 1.8;">
        <li><strong>Preencha todos os campos:</strong> Área de interesse, habilidades atuais, objetivo principal, horas disponíveis por semana, tempo estimado e estilos de aprendizagem</li>
        <li><strong>Gere seu PDI:</strong> Clique em "🚀 Gerar PDI" para criar seu plano personalizado usando IA</li>
        <li><strong>Salve o resultado:</strong> Após a geração, clique em "💾 Salvar PDI" para baixar os arquivos</li>
        <li><strong>Formatos disponíveis:</strong> O PDI será salvo em formato TXT (leitura fácil) e JSON (processamento de dados)</li>
    </ol>

    <h4>💡 Dicas para melhores resultados:</h4>
    <ul style="line-height: 1.6;">
        <li>Seja específico ao descrever suas habilidades atuais</li>
        <li>Defina objetivos claros e mensuráveis</li>
        <li>Selecione múltiplos estilos de aprendizagem para um plano mais rico</li>
        <li>Seja realista com o tempo disponível e prazo estimado</li>
    </ul>
</div>
"""))